<h1>Importing important packages</h1>

In [ ]:
import os          #For directory changing and file handling
import subprocess  #For using Powershell inside
import glob        #For returing file paths that matches a common pattern
import time        #For returning time passed
import shutil      #For copying the final file into a folder

<h3>Chaging the direcotry using OS</h3>

In [ ]:
dir_name = input()       #Input the path of your desired directory
os.chdir(str(dir_name))
os.getcwd()              #To check if you have the correct directory

<h2>Calculating parameters</h2>


In [ ]:
# * Metal Block Size *
block_x = input("x-direction: ")
block_y = input("y-direction: ")
block_z = input("z-direction: ")

metal_name = input       #Input shorthand string for the metal name, eg, for Copper it is 'Cu'
metal_lattice = input()  #Input the lattice parameter of your desired metal matrix

# * Graphene Block Size *
# * This will generally be matched with the x and y value of Platinum *

graphene_x_lattice = 2.44
graphene_y_lattice = 4.23

In [ ]:
# Graphene values for duplicating

Graphene_x = int(block_x) / graphene_x_lattice
Graphene_y = int(block_y) / graphene_y_lattice

# Metal values for duplicating

Metal_x = int(block_x) / pt_lattice
Metal_y = int(block_y) / pt_lattice
Metal_z = int(block_z) / pt_lattice

<h3>Designing the Graphene sheet</h3>
<p>Here the Graphene sheet will be extended in x and y direction by a lattice spacing of 2.44 and 4.23 correspondingly</p>
<p>We need to input the x and y value by which the sheet will be expanded</p>

In [ ]:
subprocess.run('atomsk --create graphite 2.44 4.23 C -cut below 0.2 Z -orthogonal-cell g_1.cfg', shell = True)
g_str = 'atomsk g_1.cfg -duplicate ' + str(round(Graphene_x)) + ' ' + str(round(Graphene_y)) + ' 0 g_2.cfg'
subprocess.run(g_str, shell = True)
subprocess.run('atomsk g_2.cfg -center com g_base.cfg')

<h3>Designing the Metal matrix</h3>

In [ ]:
p_str = 'atomsk --create fcc ' + str(metal_lattice) + ' ' + metal_name + ' -duplicate ' + str(round(Metal_x)) + str(' ') + str(round(Metal_y)) + str(' ') + str(round(Metal_z)) + ' pt_1.cfg'
subprocess.run(p_str, shell = True)
subprocess.run('atomsk pt_1.cfg -center com pt_base.cfg')

<h3>Completing the Nanocomposite</h3>

In [ ]:
g_layer = input("Number of Graphene layers: ")

In [ ]:
subprocess.run('atomsk --merge z 3 pt_base.cfg g_base.cfg pt_base.cfg ini_NC.cfg')
subprocess.run('atomsk --merge z 2 g_base.cfg pt_base.cfg duplicator.cfg')
if int(g_layer) == 1 :
    os.rename('ini_NC.cfg', 'NC.cfg')
else:
    os.rename('ini_NC.cfg', 'NC1.cfg')
    for layer in range(int(g_layer) - 1):
        layer_str = 'atomsk --merge z 2 NC' + str(layer+1) + '.cfg duplicator.cfg NC' + str(layer+2) + '.cfg'
        subprocess.run(layer_str, shell = True)
    file_name = 'NC' + str(g_layer) + '.cfg'
    os.rename(file_name, 'NC.cfg')

cut_x = 'atomsk NC.cfg -cut above ' + str(block_x) + ' x NC_x.cfg'
cut_y = 'atomsk NC_x.cfg -cut above ' + str(block_y) + ' y NC_c.cfg'
    
subprocess.run(cut_x, shell = True)
subprocess.run(cut_y, shell = True)
subprocess.run('atomsk NC_c.cfg -center com F_NC.cfg', shell = True)

<h3>Removing Unnecessary Files</h3> 
<p>Now we would like to keep the Final design file only and remove every other files</p>

In [ ]:
design_file = dir_name + '/F_NC.cfg'

while not os.path.exists(design_file):
    time.sleep(10)
    
if os.path.isfile(design_file):
    #Verifies FINAL DESIGN file was created, then deletes unneeded files.
    for CleanUp in glob.glob(dir_name + '/*.*'):
        #print(CleanUp)
        if not CleanUp.endswith('F_NC.cfg'):    
            os.remove(CleanUp)

In [ ]:
#Renaming the file so that finding it can be easier

design_file_name = 'NC' + str(g_layer) + 'layers.cfg'
os.rename('F_NC.cfg', design_file_name)

design_folder_name = 'NC' + str(g_layer) + 'layers'
os.mkdir(design_folder_name)

source = dir_name + "\\" + design_file_name
destination = dir_name + "\\" + design_folder_name + "\\" + design_file_name

shutil.copyfile(source, destination)
